In [2]:
## Standard libraries
import time
import json
import math
import numpy as np
import scipy
from scipy.linalg import fractional_matrix_power as frac_mat_pow

## Imports for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

def setup_gpu(seed=42):
    # Function for setting the seed
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): # GPU operation have separate seed
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # Additionally, some operations on a GPU are implemented stochastic for efficiency
    # We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Fetching the device that will be used throughout this notebook
    device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
    print("Using device -> ", device)



In [3]:
N = 5
T = 10
d = 2
Sigma = torch.eye(d)
Lambda = 0
memory = 0
X = [] 
Drift = []
Diffusion = []
input_output_pairs = []

In [4]:
Z = torch.randn(N, d)
B = torch.bernoulli(torch.tensor([0.5]*N))

In [5]:
long_term_mean = 0.01
mean_reversion = 0.5
volitality = 0.001

def mu(X):
    return (long_term_mean - X) * mean_reversion

def varsigma(X):
    return torch.ones(X.shape) * volitality

In [6]:
for t in range(T+1):
    if t == 0:
        X.append(torch.zeros(N, d)) # X[-1]
        Drift.append(torch.zeros(N, d))
        Diffusion.append(torch.zeros(N, d))
        X.append(torch.zeros(N, d)) # X[0]
        Drift.append(torch.zeros(N, d)) 
        Diffusion.append(torch.zeros(N, d))
    else:
        Z = torch.randn(N, d)
        B = torch.bernoulli(torch.tensor([0.5]*N))
        drift_t = memory * mu(X[t-2]) + (1-memory) * mu(X[t-1])
        S = torch.einsum("n,dk->nk", (B * Lambda), torch.eye(d))
        diffusion_t = S + torch.einsum("nd,dk->nk", varsigma(X[t-1]), Sigma)
        x_t = X[t-1] + drift_t + (diffusion_t * Z)
        X.append(x_t)
        Drift.append(drift_t)
        Diffusion.append(diffusion_t)
        

In [7]:
for i in range(2,T+2):
    print("\nX[i] -> ", X[i], "\nDrift[i] -> ", Drift[i], "\nDiffusion[i] -> ", Diffusion[i])


X[i] ->  tensor([[0.0054, 0.0050],
        [0.0051, 0.0054],
        [0.0052, 0.0043],
        [0.0055, 0.0051],
        [0.0054, 0.0057]]) 
Drift[i] ->  tensor([[0.0050, 0.0050],
        [0.0050, 0.0050],
        [0.0050, 0.0050],
        [0.0050, 0.0050],
        [0.0050, 0.0050]]) 
Diffusion[i] ->  tensor([[0.0010, 0.0010],
        [0.0010, 0.0010],
        [0.0010, 0.0010],
        [0.0010, 0.0010],
        [0.0010, 0.0010]])

X[i] ->  tensor([[0.0045, 0.0056],
        [0.0062, 0.0056],
        [0.0057, 0.0049],
        [0.0065, 0.0049],
        [0.0050, 0.0042]]) 
Drift[i] ->  tensor([[0.0050, 0.0050],
        [0.0050, 0.0050],
        [0.0050, 0.0050],
        [0.0050, 0.0050],
        [0.0050, 0.0050]]) 
Diffusion[i] ->  tensor([[0.0010, 0.0010],
        [0.0010, 0.0010],
        [0.0010, 0.0010],
        [0.0010, 0.0010],
        [0.0010, 0.0010]])

X[i] ->  tensor([[0.0070, 0.0075],
        [0.0092, 0.0077],
        [0.0062, 0.0080],
        [0.0082, 0.0082],
        [0.0098,

In [8]:
X_NTd = torch.stack(X, dim=1)
Drift_NTd = torch.stack(Drift, dim=1)
Diffusion_NTd = torch.stack(Diffusion, dim=1)


In [9]:
X_NTd[:, 2:, :][1]

tensor([[0.0051, 0.0054],
        [0.0062, 0.0056],
        [0.0092, 0.0077],
        [0.0079, 0.0048],
        [0.0111, 0.0102],
        [0.0092, 0.0085],
        [0.0094, 0.0103],
        [0.0096, 0.0084],
        [0.0093, 0.0121],
        [0.0080, 0.0090]])

In [10]:
X_NTd[1]

tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0051, 0.0054],
        [0.0062, 0.0056],
        [0.0092, 0.0077],
        [0.0079, 0.0048],
        [0.0111, 0.0102],
        [0.0092, 0.0085],
        [0.0094, 0.0103],
        [0.0096, 0.0084],
        [0.0093, 0.0121],
        [0.0080, 0.0090]])

In [11]:

X = X_NTd[:, 2:, :]
Drift = Drift_NTd[:, 2:, :]
Diffusion = Diffusion_NTd[:, 2:, :]